# Init the enivorment

# test with one collections

In [8]:
from scripts.build_index import load_index, search
from scripts.reranker import load_reranker, merge_results, rerank, _load_chunk_texts
from sentence_transformers import SentenceTransformer

COLLECTION = "A"       
QUERY      = "Which Pittsburgh bridge is the longest?"   

model = SentenceTransformer("BAAI/bge-m3")

reranker = load_reranker()

Loading weights: 100%|██████████| 391/391 [00:00<00:00, 1779.16it/s, Materializing param=pooler.dense.weight]                               


  Reranker        : BAAI/bge-reranker-v2-m3  (本地推理)
  Device          : mps


Loading weights: 100%|██████████| 393/393 [00:00<00:00, 1459.10it/s, Materializing param=roberta.encoder.layer.23.output.dense.weight]              


In [9]:
idx, meta = load_index(COLLECTION)
chunk_texts = _load_chunk_texts(COLLECTION)

dense_hits = search(QUERY, idx, meta, model, top_k=20)
print(f" {len(dense_hits)} been search")

  Loaded index: dim=1024  ntotal=592
 20 been search


In [10]:
# rerank
candidates = merge_results(dense_hits, sparse_hits=[])
results    = rerank(QUERY, candidates, reranker, chunk_texts, top_n=5)

print(f"\n{'Rank':<6} {'Rerank':>8}  {'Dense':>8}  chunk_id")
print("-" * 55)
for r in results:
    print(f"  #{r['rerank_rank']:<4} {r['rerank_score']:>8.4f}  {r['dense_score']:>8.4f}  {r['chunk_id']}")


Rank     Rerank     Dense  chunk_id
-------------------------------------------------------
  #1      0.3885    0.3860  A_wiki_Andrew Carnegie - Wikipedia__0008
  #2      0.0046    0.3808  A_wiki_Andrew Carnegie - Wikipedia__0000
  #3      0.0024    0.4011  A_wiki_Carnegie Library of Pittsburgh - Wikipedia__0001
  #4      0.0023    0.4227  A_wiki_Shannopin's Town - Wikipedia__0000
  #5      0.0020    0.4674  A_wiki_Carnegie Mellon University traditions - Wikipedia__0002
